# Telegram Cryptocurrency Analysis

* Analysis of trends in top Telegram Crypto Groups. 

* Adapted from kaggle tutorial: https://www.kaggle.com/aagghh/trends-in-crypto-space-telegram-analysis

In [4]:
# import libraries
import numpy as np
import pandas as pd 
import plotly
from plotly import graph_objs as go
import datetime

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from plotly import tools
import seaborn as sns
init_notebook_mode(connected=True)
from itertools import zip_longest
import string 
import re

from nltk.corpus import stopwords 
from nltk.util import ngrams
import nltk 

In [28]:
# obtain stopwords from nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\richa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
# set filepaths for the datasets
# this will be different for you so set the filepath to where you have stored the datasets

dataset1 = 'C://Users/richa/Documents/Coding+/SPR21/csce 5300 (big data)/telegram analysis/group_messages_binance.json/group_messages_binance.json'
dataset2 = 'C://Users/richa/Documents/Coding+/SPR21/csce 5300 (big data)/telegram analysis/group_messages_bittrex.json/group_messages_bittrex.json'
dataset3 = 'C://Users/richa/Documents/Coding+/SPR21/csce 5300 (big data)/telegram analysis/group_messages_huobi.json/group_messages_huobi.json'
dataset4 = 'C://Users/richa/Documents/Coding+/SPR21/csce 5300 (big data)/telegram analysis/group_messages_kucoin.json/group_messages_kucoin.json'
dataset5 = 'C://Users/richa/Documents/Coding+/SPR21/csce 5300 (big data)/telegram analysis/group_messages_okex.json/group_messages_okex.json'

In [14]:
# upload datasets
# be careful about size of files when running cells - might crash your session if running on colab

binance = pd.read_json(dataset1)

In [17]:
bittrex = pd.read_json(dataset2)


In [18]:
huobi = pd.read_json(dataset3)

In [19]:
kucoin = pd.read_json(dataset4)


In [20]:
okex = pd.read_json(dataset5)

In [16]:
# check data real quick

binance.head()

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,action
0,Message,10072598,"{'_': 'PeerChannel', 'channel_id': 1146170349}",2021-01-29 07:16:46+00:00,BTC rekt down,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
1,Message,10072597,"{'_': 'PeerChannel', 'channel_id': 1146170349}",2021-01-29 07:16:44+00:00,If XVG is pumped. It's alt season,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
2,Message,10072596,"{'_': 'PeerChannel', 'channel_id': 1146170349}",2021-01-29 07:16:44+00:00,👍🚀,False,False,False,False,False,...,None,[],NaN,NaN,None,None,NaN,NaN,[],NaN
3,Message,10072595,"{'_': 'PeerChannel', 'channel_id': 1146170349}",2021-01-29 07:16:43+00:00,Imagine if you bought the top,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
4,Message,10072594,"{'_': 'PeerChannel', 'channel_id': 1146170349}",2021-01-29 07:16:42+00:00,Just how many people get rekted from doge 0.08...,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN


In [21]:
bittrex.head() 

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,action
0,Message,154587,"{'_': 'PeerChannel', 'channel_id': 1478738228}",2021-01-27 14:17:10+00:00,Any updates about skm trading event reward??\n...,False,False,False,False,False,...,None,[],NaN,NaN,None,None,NaN,NaN,[],NaN
1,Message,154586,"{'_': 'PeerChannel', 'channel_id': 1478738228}",2021-01-27 14:05:48+00:00,‼️SCAM ALERT! Bittrex US / Bittrex Global seni...,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
2,Message,154585,"{'_': 'PeerChannel', 'channel_id': 1478738228}",2021-01-27 13:52:33+00:00,I have a certain amount of BTXCRD on my accoun...,False,False,False,False,False,...,None,"[{'_': 'MessageEntityUrl', 'offset': 189, 'len...",NaN,NaN,None,2021-01-27T13:52:42+00:00,NaN,NaN,[],NaN
3,Message,154584,"{'_': 'PeerChannel', 'channel_id': 1478738228}",2021-01-27 13:36:51+00:00,More coin,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
4,Message,154581,"{'_': 'PeerChannel', 'channel_id': 1478738228}",2021-01-27 13:27:09+00:00,What is this for a Kindergarten here? I need ...,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN


In [22]:
huobi.head()

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,action
0,Message,1018295,"{'_': 'PeerChannel', 'channel_id': 1119054974}",2021-01-27 12:18:20+00:00,"I'm confused about how to find my balance, whi...",False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
1,Message,1018294,"{'_': 'PeerChannel', 'channel_id': 1119054974}",2021-01-27 12:15:16+00:00,⚠️ Important Reminder ☠️\n\n🚫 Huobi Global doe...,False,False,False,False,False,...,None,"[{'_': 'MessageEntityUrl', 'offset': 327, 'len...",NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
2,Message,1018293,"{'_': 'PeerChannel', 'channel_id': 1119054974}",2021-01-27 11:52:45+00:00,Until when is the Kyc video review?,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
3,Message,1018288,"{'_': 'PeerChannel', 'channel_id': 1119054974}",2021-01-27 11:42:54+00:00,"Hello there welcome, how may I help you today?",False,False,False,False,False,...,None,[],NaN,NaN,None,None,NaN,NaN,[],NaN
4,Message,1018287,"{'_': 'PeerChannel', 'channel_id': 1119054974}",2021-01-27 11:42:32+00:00,No,False,False,False,False,False,...,None,[],NaN,NaN,None,None,NaN,NaN,[],NaN


In [23]:
kucoin.head()

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,action
0,Message,2573626,"{'_': 'PeerChannel', 'channel_id': 1080116228}",2021-01-28 11:31:28+00:00,SHR(SHR) \n Price: $ 0.015741\n Change: (📉-0.6...,False,False,False,False,False,...,"{'_': 'ReplyInlineMarkup', 'rows': [{'_': 'Key...",[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
1,Message,2573625,"{'_': 'PeerChannel', 'channel_id': 1080116228}",2021-01-28 11:31:27+00:00,Shr?,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
2,Message,2573624,"{'_': 'PeerChannel', 'channel_id': 1080116228}",2021-01-28 11:31:15+00:00,Good day to everyone,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
3,Message,2573623,"{'_': 'PeerChannel', 'channel_id': 1080116228}",2021-01-28 11:30:41+00:00,We'll relay your feedback with the team,False,False,False,False,False,...,None,[],NaN,NaN,None,None,NaN,NaN,[],NaN
4,Message,2573622,"{'_': 'PeerChannel', 'channel_id': 1080116228}",2021-01-28 11:30:09+00:00,Please help my fund withdraw,False,False,False,False,False,...,None,[],NaN,NaN,None,None,NaN,NaN,[],NaN


In [24]:
okex.head()

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,action
0,Message,1406637,"{'_': 'PeerChannel', 'channel_id': 1450606717}",2021-01-27 09:35:20+00:00,Hi Michael J\n\nWelcome to OKEx Community! \n...,False,False,False,False,False,...,"{'_': 'ReplyInlineMarkup', 'rows': [{'_': 'Key...","[{'_': 'MessageEntityMentionName', 'offset': 3...",NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
1,Message,1406635,"{'_': 'PeerChannel', 'channel_id': 1450606717}",2021-01-27 09:35:08+00:00,Rising again so sharply !!!,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
2,Message,1406634,"{'_': 'PeerChannel', 'channel_id': 1450606717}",2021-01-27 09:35:03+00:00,"‼️Dear users, Please be informed that due to h...",False,False,False,False,False,...,None,"[{'_': 'MessageEntityUnderline', 'offset': 45,...",NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
3,Message,1406633,"{'_': 'PeerChannel', 'channel_id': 1450606717}",2021-01-27 09:34:29+00:00,find a good shot,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN
4,Message,1406632,"{'_': 'PeerChannel', 'channel_id': 1450606717}",2021-01-27 09:34:26+00:00,Yaaaaaayyyyyy,False,False,False,False,False,...,None,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",None,NaN,NaN,[],NaN


In [26]:
# consolidating datasets into a dataframe

huobi['Channel_name'] = 'Huobi'
okex['Channel_name'] = 'OKEx'
bittrex['Channel_name'] = 'Bittrex'
binance['Channel_name'] = 'Binance'
kucoin['Channel_name'] = 'Kucoin'

consolidated_data = huobi
consolidated_data = consolidated_data.append(okex)
consolidated_data = consolidated_data.append(bittrex)
consolidated_data = consolidated_data.append(binance)
consolidated_data = consolidated_data.append(kucoin)

In [29]:
#date manipulation
stopwords = stopwords.words('english')
consolidated_data['CreationDate'] = pd.to_datetime(consolidated_data['date'])
consolidated_data['CreationYear'] = consolidated_data['CreationDate'].dt.year
consolidated_data['CreationMonth'] = consolidated_data['CreationDate'].dt.month
consolidated_data['CreationMonth'] = consolidated_data['CreationMonth'].apply(lambda x : "0"+str(x) if len(str(x)) < 2 else x)
consolidated_data['CreationDay'] = "27"
consolidated_data['MessageDate'] = consolidated_data["CreationYear"].astype(str) +"-"+ consolidated_data["CreationMonth"].astype(str) +"-"+ consolidated_data["CreationDay"].astype(str)
consolidated_data['Message'] = consolidated_data['message'].fillna(" ")

## cleaning text
def clntxt(text):
    text = text.lower()
    text = " ".join([c for c in text.split() if c not in stopwords])
    for c in string.punctuation:
        text = text.replace(c, " ")
    text = " ".join([c for c in text.split() if c not in stopwords])
    
    words = []
    ignorewords = ["www", "http","https" "com"]
    for wrd in text.split():
        if len(wrd) <= 2: 
            continue
        if wrd in ignorewords:
            continue
        words.append(wrd)
    text = " ".join(words)    
    return text

# counting mentions
def count_presence(txt, wrds):    
    cnt = 0
    txt = " "+txt+" "
    for wrd in wrds.split("|"):
        if " "+wrd+" " in txt:
            cnt += 1 
    return cnt

consolidated_data['CleanMessage'] = consolidated_data['Message'].apply(lambda x : clntxt(x))

## Bitcoin vs. Ethereum

In [30]:
#plotting telegram mentions

def plotit(listed, title):    
    traces = []
    for model in listed:
        temp = consolidated_data.groupby('MessageDate').agg({model : "sum"}).reset_index()
        trace = go.Scatter(x = temp["MessageDate"], y = temp[model], name=model.split("|")[0].title(), line=dict(shape="spline", width=2), mode = "lines")
        traces.append(trace)

    layout = go.Layout(
        paper_bgcolor='#fff',
        plot_bgcolor="#fff",
        legend=dict(orientation="h", y=1.1),
        title=title,
        title_x=0.5,
        xaxis=dict(
            gridcolor='rgb(255,255,255)',
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False
        ),
        yaxis=dict(
            title="Number of Mentions",
            gridcolor='rgb(255,255,255)',
            showgrid=False,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False
        ),
    )

    fig = go.Figure(data=traces, layout=layout)
    iplot(fig)

#btc vs eth
models = ["btc", "eth"]
for col in models:
    consolidated_data[col] = consolidated_data["CleanMessage"].apply(lambda x : count_presence(x, col))
plotit(models, "Telegram Discussions: BTC vs ETH")    

## DOT vs XRP vs LTC vs XLM

In [31]:
#other popular crypto-currencies
models = ["dot", "xrp", "ltc", "xlm"]
for col in models:
    consolidated_data[col] = consolidated_data["CleanMessage"].apply(lambda x : count_presence(x, col))
plotit(models, "Telegram Discussions: DOT vs XRP vs LTC vs XLM")    

## BTC vs DeFi

In [32]:
#btc vs defi
models = ["btc", "defi"]
for col in models:
    consolidated_data[col] = consolidated_data["CleanMessage"].apply(lambda x : count_presence(x, col))
plotit(models, "Telegram Discussions: BTC vs DeFi")    

## USDT (Centralized Stablecoins) vs DAI (Decentralized)

In [33]:
#stablecoins
models = ["usdt", "dai"]
for col in models:
    consolidated_data[col] = consolidated_data["CleanMessage"].apply(lambda x : count_presence(x, col))
plotit(models, "Stable coins: USDT vs DAI") 

## Decentralized Exchanges (UNISWAP vs Sushiswap vs 1inch)

In [34]:
#DEXs
models = ["uniswap", "sushiswap", "1inch"]
for col in models:
    consolidated_data[col] = consolidated_data["CleanMessage"].apply(lambda x : count_presence(x, col))
plotit(models, "DEX: UNISWAP vs Sushiswap vs 1inch")

## DeFi Protocols (MakerDAO vs Compound)

In [35]:
#DEXs
models = ["uniswap", "sushiswap", "1inch"]
for col in models:
    consolidated_data[col] = consolidated_data["CleanMessage"].apply(lambda x : count_presence(x, col))
plotit(models, "DEX: UNISWAP vs Sushiswap vs 1inch")